In [1]:
def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Record start time
        result = func(*args, **kwargs)
        end_time = time.time()    # Record end time
        print(f"Function '{func.__name__}' executed in {end_time - start_time:.4f} seconds")
        return result
    return wrapper

In [2]:
import numpy as np
import xarray as xr
import rasterio
from rasterio.transform import from_origin
from pyproj import Transformer
from scipy.interpolate import griddata
from pathlib import Path
import time

# --- Open the NetCDF file ---
nc_path = Path("/home/data/sst/F002_L1__IR__L2L1M0__2025-02-04T235830.003911Z_2025-04-10T200213.364384Z_c395dd14.nc4")

# Method 1 (scipy)

In [14]:
@timing_decorator
def nc2gtiff_scipy(inp_nc: str = "/home/data/sst/F002_L1__IR__L2L1M0__2025-02-04T235830.003911Z_2025-04-10T200213.364384Z_c395dd14.nc4",
                   out_tiff: str = "/home/data/sst/bt_L2_epsg2100_scipy.tif",
                   res: int = 150):
    
    # --- Open the NetCDF file ---
    nc_path = Path(inp_nc)
    
    ds = xr.open_dataset(nc_path)

    # --- Example: select brightness temperature variable ---
    # Replace "BT" with the actual name in your file
    bt = ds["brightness_temperature"].isel(bands=0).values
    lat = ds['lat'].values
    lon = ds['lon'].values
    flags = ds['flags'].values

    # Convert Kelvin to Celsius (optional)
    bt = bt - 273.15

    # Flatten arrays
    lat_flat = lat.flatten()
    lon_flat = lon.flatten()
    bt_flat = bt.flatten()

    # Remove NaN or fill values
    mask = ~np.isnan(bt_flat)
    lat_flat = lat_flat[mask]
    lon_flat = lon_flat[mask]
    bt_flat = bt_flat[mask]

    # --- Transform coordinates to EPSG:2100 ---
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:2100", always_xy=True)
    x_proj, y_proj = transformer.transform(lon_flat, lat_flat)

    # --- Define regular target grid in EPSG:2100 ---
    # Choose resolution in meters (e.g., 1000 m)

    xmin, xmax = np.min(x_proj), np.max(x_proj)
    ymin, ymax = np.min(y_proj), np.max(y_proj)

    # Compute grid
    nx = int((xmax - xmin) / res)
    ny = int((ymax - ymin) / res)

    grid_x, grid_y = np.meshgrid(
        np.linspace(xmin, xmax, nx),
        np.linspace(ymax, ymin, ny)  # note: y goes from top to bottom
    )

    # --- Interpolate onto the grid ---
    bt_grid = griddata(
        (x_proj, y_proj),
        bt_flat,
        (grid_x, grid_y),
        method="linear"
    )

    # Fill NaNs with nodata
    nodata = -9999
    bt_grid = np.nan_to_num(bt_grid, nan=nodata)

    # --- Define raster transform ---
    transform = from_origin(xmin, ymax, res, res)

    # --- Save to GeoTIFF ---
    with rasterio.open(
        out_tiff,
        "w",
        driver="GTiff",
        height=bt_grid.shape[0],
        width=bt_grid.shape[1],
        count=1,
        dtype=bt_grid.dtype,
        crs="EPSG:2100",
        transform=transform,
        nodata=nodata,
    ) as dst:
        dst.write(bt_grid, 1)

    print(f"Saved: {out_tiff}")


In [15]:
nc2gtiff_scipy()

Saved: /home/data/sst/bt_L2_epsg2100_scipy.tif
Function 'nc2gtiff_scipy' executed in 22.9902 seconds


# Method 2 (pyresample)

In [3]:
from pyproj import CRS
from pyproj import Transformer
from pyresample import geometry, kd_tree

In [4]:
@timing_decorator
def nc2gtiff_pyresample(inp_nc: str = "/home/data/sst/F002_L1__IR__L2L1M0__2025-02-04T235830.003911Z_2025-04-10T200213.364384Z_c395dd14.nc4",
                   out_tiff: str = "/home/data/sst/bt_L2_epsg2100_pyresample.tif",
                   res: int = 150):

    # --- Open the NetCDF file ---
    nc_path = Path(inp_nc)

    ds = xr.open_dataset(nc_path)

    # --- Example: select brightness temperature variable ---
    # Replace "BT" with the actual name in your file
    bt = ds["brightness_temperature"].isel(bands=0).values
    lat = ds['lat'].values
    lon = ds['lon'].values
    flags = ds['flags'].values

    # Convert Kelvin to Celsius (optional)
    bt = bt - 273.15

    # Define swath geometry (source data)
    swath = geometry.SwathDefinition(lons=lon, lats=lat)

    # Define target area in EPSG:2100
    # Choose resolution in meters (e.g. 1000 m)
    xmin, xmax = lon.min(), lon.max()
    ymin, ymax = lat.min(), lat.max()

    # Get projected extent in EPSG:2100
    crs_src = CRS.from_epsg(4326)
    crs_dst = CRS.from_epsg(2100)

    # Transform extent
    transformer = Transformer.from_crs(crs_src, crs_dst, always_xy=True)
    x_min, y_min = transformer.transform(xmin, ymin)
    x_max, y_max = transformer.transform(xmax, ymax)

    # Build target grid
    nx = int((x_max - x_min) / res)
    ny = int((y_max - y_min) / res)

    area_def = geometry.AreaDefinition(
        "target_epsg2100",
        "Reprojected Area",
        "epsg2100",
        crs_dst.to_wkt(),
        nx,
        ny,
        (x_min, y_min, x_max, y_max)
    )

    # Resample swath to grid (nearest or bilinear)
    bt_grid = kd_tree.resample_nearest(
        swath, bt, area_def,
        radius_of_influence=5000,  # meters
        fill_value=-9999
    )

    # --- Save to GeoTIFF ---
    transform = from_origin(x_min, y_max, res, res)
    with rasterio.open(
        out_tiff,
        "w",
        driver="GTiff",
        height=ny,
        width=nx,
        count=1,
        dtype=bt_grid.dtype,
        crs="EPSG:2100",
        transform=transform,
        nodata=-9999
    ) as dst:
        dst.write(bt_grid, 1)

    print(f"Saved: {out_tiff}")

In [5]:
nc2gtiff_pyresample()

Saved: /home/data/sst/bt_L2_epsg2100_pyresample.tif
Function 'nc2gtiff_pyresample' executed in 2.7751 seconds


# Method 3 (Dask) (erroneous output)

In [6]:
from pyproj import CRS, Transformer
from pyresample import geometry, kd_tree
from dask import array as da
from dask.diagnostics import ProgressBar

In [7]:
@timing_decorator
def nc2gtiff_dask(inp_nc: str = "/home/data/sst/F002_L1__IR__L2L1M0__2025-02-04T235830.003911Z_2025-04-10T200213.364384Z_c395dd14.nc4",
                   out_tiff: str = "/home/data/sst/bt_L2_epsg2100_dask.tif",
                   res: int = 150):

    # --- Open NetCDF file with Dask ---
    nc_path = Path(inp_nc)

    ds = xr.open_dataset(nc_path, chunks={"rows": 1000, "columns": 1000})

    # Extract brightness_temperature band 1 lazily
    bt = ds["brightness_temperature"].isel(bands=0).data  # dask array
    lat = ds["lat"].data
    lon = ds["lon"].data

    # Convert Kelvin to Celsius lazily
    bt = bt - 273.15  

    # Define swath geometry
    swath = geometry.SwathDefinition(lons=lon, lats=lat)

    # --- Define target area in EPSG:2100 ---
    # Compute bounds in EPSG:2100
    crs_src = CRS.from_epsg(4326)
    crs_dst = CRS.from_epsg(2100)
    transformer = Transformer.from_crs(crs_src, crs_dst, always_xy=True)

    lon_min, lon_max = float(ds["lon"].min()), float(ds["lon"].max())
    lat_min, lat_max = float(ds["lat"].min()), float(ds["lat"].max())

    x_min, y_min = transformer.transform(lon_min, lat_min)
    x_max, y_max = transformer.transform(lon_max, lat_max)

    nx = int((x_max - x_min) / res)
    ny = int((y_max - y_min) / res)

    area_def = geometry.AreaDefinition(
        "target_epsg2100",
        "Reprojected Area",
        "epsg2100",
        crs_dst.to_wkt(),
        nx,
        ny,
        (x_min, y_min, x_max, y_max)
    )

    # --- Resample lazily with Dask ---
    def resample_block(bt_block, lat_block, lon_block):
        """Resample one chunk of data into EPSG:2100 grid."""
        swath_chunk = geometry.SwathDefinition(lons=lon_block, lats=lat_block)
        return kd_tree.resample_nearest(
            swath_chunk, bt_block, area_def,
            radius_of_influence=450,  # meters
            fill_value=-9999
        )

    # Apply chunk-wise (use dask.map_blocks)
    bt_resampled = da.map_blocks(
        resample_block, bt, lat, lon,
        dtype=np.float32, drop_axis=[]
    )

    # Trigger computation with progress bar
    with ProgressBar():
        bt_grid = bt_resampled.compute()

    # --- Save to GeoTIFF ---
    transform = from_origin(x_min, y_max, res, res)
    with rasterio.open(
        out_tiff,
        "w",
        driver="GTiff",
        height=ny,
        width=nx,
        count=1,
        dtype=bt_grid.dtype,
        crs="EPSG:2100",
        transform=transform,
        nodata=-9999
    ) as dst:
        dst.write(bt_grid, 1)

    print(f"Saved: {out_tiff}")

In [8]:
nc2gtiff_dask()

/tmp/ipykernel_1730/2402310149.py:9: UserWarning: The specified chunks separate the stored chunks along dimension "columns" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(nc_path, chunks={"rows": 1000, "columns": 1000})
/tmp/ipykernel_1730/2402310149.py:9: UserWarning: The specified chunks separate the stored chunks along dimension "rows" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(nc_path, chunks={"rows": 1000, "columns": 1000})


[#####################                   ] | 54% Completed | 101.87 ms

/opt/venv/lib/python3.12/site-packages/pyresample/kd_tree.py:261: UserWarning: Searching for 1 neighbours in 0 data points
  get_neighbour_info(source_geo_def,


[########################################] | 100% Completed | 2.98 sms
Saved: /home/data/sst/bt_L2_epsg2100_dask.tif
Function 'nc2gtiff_dask' executed in 3.5584 seconds
